----------------------
# Closed vs Open Models


Proprietary model trained on vast amounts of data at considerable expense. Some companies have released smaller version of these models under different open frameworks and licenses


## Closed/Proprietary LLMs

OpenAI’s ChatGPT or Anthropic’s Claude are examples of proprietary systems where the public can’t access the code and model weights

Closed source LLMs are language models where the model source or weights are not publicly available. Often developed by companies with significant resource for development and improvement. 
* Access may be restricted or a paid subsciption
* Customisation may be limited as access to underlaying code and architecture mat be limit. Limiting customization to fine-tuning on pre-trained models and parmeter configuration
* Vendor Support
* Proprietary licensing
* Advanced Features and Performance. Proprietary models usually maximum performance

eg: OpenAI ChatGPT, Google Gemini, Antropic Claude, Cohere Command


## Open LLM

Most of the top performing open models are derived from closed models or much large models

Popular LLM Families
* OpenAI GPT family
* Meta LLaMA family
* Google PaLM family


bfgb
* Leaked LLama wieghts - Mar '23
* Leaked Google document: [We Have No Moat, And Neither Does OpenAI](https://www.semianalysis.com/p/google-we-have-no-moat-and-neither)
 * Open models going thru' very quick iterations (every 1~2 wks)
* Community is driving innovaiton
  * Quantisation
  * LORa



Open models are freely available to the public. This fosters innovation, collaboration, and community driven development
* Customizability
* Support is via the community. This can be large community of developers depending on the model
* Licensing needs to be reviewed if commercial use or research use
* Transparent development which can help build trust
* Availability of high-quality open base models (LLaMA, Mistral 
* Fewer parameters: 3B, 13B, 30B, 70B
  
eg. LLaMA family, Mistral, DBRX


### Open source vs open weight

The core question is whether simply releasing a model’s weights while keeping training methodology and data proprietary can be considered true open sourcing. 



Releasing only a model's weights broadly enables application development but concentrates control among a small group of organizations. Enabling open source access distributes control but requires greater commitment to transparency and decentralization.

* Open source
  * releasing a model as open source would entail providing the full source code and information required for retraining the model from scratch. This includes the model architecture code, training methodology and hyperparameters, the original training dataset, documentation, and other relevant details.
  * open source enables model understanding and customization but requires substantially more work to release.
  * [Olmo](https://allenai.org/olmo)


* Open weights isn’t open source unless they provide full access to their training set and source code.
  * Open weights refers to releasing only the pretrained parameters or weights of the neural network model itself.
  * This allows others to use the model for inference and fine-tuning.
  * However, the training code, original dataset, model architecture details, and training methodology is not provided.
  * open weights allows model use but not full transparency



![Open vs close model ELO](./data/arena_elo.jpg)



-----------------
References:

* https://www.linkedin.com/pulse/deep-dive-opensource-llms-vs-proprietor-dr-rabi-prasad-kutuc/
* https://promptengineering.org/llm-open-source-vs-open-weights-vs-restricted-weights/




-----------------------
# Popular open models


## LLaMa Family

* LLaMA (Large Language Model Meta AI) are a family of LLM models release by [Meta](https://ai.meta.com/blog/meta-llama-3/). Originally model released to researchers under non-commercial license (Mar '23) however model weights were leaked.
* A large number of researchers have extended LLaMA models by either instruction tuning or continual pretraining
  *  instruction tuning LLaMA has become a major approach to developing customized or specialized models, due to the relatively low computational costs.
* LLaMA is available on HuggingFace [Meta profile](https://huggingface.co/meta-llama)


![LLaMA](./data/llama.png)



*  Stanford [Alpaca-52K](https://github.com/tatsu-lab/stanford_alpaca) instruction-following data generated by the techniques in the [Self-Instruct](https://github.com/yizhongw/self-instruct)
*  On the self-instruct evaluation set, Alpaca shows many behaviors similar to OpenAI’s text-davinci-003, but is also surprisingly small and easy/cheap to reproduce.
*  Alpaca: [fine-tune](https://github.com/tatsu-lab/stanford_alpaca?tab=readme-ov-file#fine-tuning)
*  LLaMMA models using standard Hugging Face training code
*  Alpaca cost about \$500 to train.
* [Vicuna](https://lmsys.org/blog/2023-03-30-vicuna/) fine-tuned LLaMa model using 70K user-shared ChatGPT conversations
  * The cost of training Vicuna-13B is around \$300.
  * Non-commercial use



---------------------
## Mistral

* Open weights NOT open source
  * Open weights isn’t open source unless they provide full access to their training set and source code. In all respect to the capabilities of Mistral’s models, it is an extreme stretch to call company that’s dropping torrents of weight binary, an OPEN SOURCE
  * Don't expose how training, recipe, how to collect the data, mixture of experts
  * Currently focused on developer experience first
  * Not just APIs --> because you need AI integrator
* Weights available on HuggingFace [Mistral AI profile](https://huggingface.co/mistralai)



----------------------
## Grok (X)

* Open weights
* Available on HuggingFace [xAI profile](https://huggingface.co/xai-org)



-----------------------
## DBRX

* DBRX is a general purpose LLM created by [Databricks](https://www.databricks.com/blog/introducing-dbrx-new-state-art-open-llm)
* Mixture of experts (MoE) architecture
  * Routrer
* DBRX is fine-grained, meaning it uses a larger number of smaller experts. DBRX has 16 experts and chooses 4, 
* Weights available on HuggingFace [Databrick profile](https://huggingface.co/databricks)


-----------------------
## References:

*


-----------------------
# Running LLMs locally


* Laptop
* No need to use external APIs
* Own cloud with GPU/
* * Groq
* Run on phones

  
## Ollama

[Analyse expenses with local LLM](https://www.youtube.com/watch?v=h_GTxRFYETY)


  * [Ollama](https://ollama.com/) <--
    * Simple and fast
    * Large range of [models](https://ollama.com/library) available
    * ```ollama pull mistral```
    * ```ollama run mistral```
    * [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)
      * ```ollama create my-model-name -f my-model-file```
      * ```ollma run my-model-name```
     

-----------------------
## GTP4ALL

  * [GPT4ALL](https://gpt4all.io/) <--
    * Chat with you private data. Can add own docs for context
    * Plug-in 
    * Limited models




-----------------------
## Hugging Face

  * HuggingFace [Transformers](https://huggingface.co/docs/transformers/en/index)
    * Need to know how to code (Python)
    * ML knowledge
   


-----------------------
## LLaMa.cpp


  * [LLaMa.cpp](https://github.com/ggerganov/llama.cpp) <--
    * Fast
    * Csan run bigger models on smaller hardware
    * Uses **GGUF** format. Modern and efficient
    * Limited model support
   

----------------------
## Run transformers in the browser

[Transformers.js](https://huggingface.co/docs/transformers.js/en/index)

Syntax 740 podcast
* Run in browser or on node server
* ONNX model format
* Microsoft **ONNX runtime**
* HuggingFace convert models to ONNX
* Run in browser
  * No server compute
  * Privacy - no data sent to servers
  * Rich JS tools to interact with browser
  * Everyone has browser
  * Soon web GPU support - speed
* Run on node JS server --> Faster





# Running LLMs locally

      
Way??
* Privacy
* Compliance
* Trust
* Cost????
* Smaller 




Locally
  * [Ollama](https://ollama.com/) <--
    * Simple and fast
    * Large range of [models](https://ollama.com/library) available
    * ```ollama pull mistral```
    * ```ollama run mistral```
    * [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)
      * ```ollama create my-model-name -f my-model-file```
      * ```ollma run my-model-name```
  * [GPT4ALL](https://gpt4all.io/) <--
    * Chat with you private data. Can add own docs for context
    * Plug-in 
    * Limited models
  * HuggingFace [Transformers](https://huggingface.co/docs/transformers/en/index)
    * Need to know how to code (Python)
    * ML knowledge
  * [LLaMa.cpp](https://github.com/ggerganov/llama.cpp) <--
    * Fast
    * Csan run bigger models on smaller hardware
    * Uses **GGUF** format. Modern and efficient
    * Limited model support
  
  * [LangChain](https://python.langchain.com/docs/integrations/platforms/)
    * Run local and remote models
    * Slow (Python)

  * [llamafile](https://github.com/Mozilla-Ocho/llamafile)
    * Embed model in executionable file and run anywhere
    * Builds on llama.cpp
    * Spped optimisation for various hardware - https://justine.lol/matmul/



### Quantisation

Reduce memory footprint for the model weights


### File formats

* Model and fine-tuning file formats
  * Uses **GGUF** format. Modern and efficient


## Other alternatives - Groq

* [Groq cloud](https://console.groq.com/playground)
  * https://simonwillison.net/2024/Apr/22/llama-3/
  * Serves at very high speeds - 800 tokens/sec
  * Groq API (key required)




-------------------------
## References:

* 6 Ways For Running A [Local LLM](https://semaphoreci.com/blog/local-llm)



------------------------
# LLM Leaderboard

* LMSYS Chatbot Arena [leaderboard](https://chat.lmsys.org/?leaderboard)
* [Benchmarking](https://chat.lmsys.org/) LLMs in the Wild
* HuggingFace [leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)



## Model Benchmarks

* AI2 reasoning challenge - grade school science questions
* HellaSwag - Common sense
* MMLU - Massive Multitask Manguage Understanding measure how diverse LLM knowledge is
* TruthfulQA - How truthful is a model
* WinoGrande - commonsense reasoning
* GSM8K - maths reasoning


---------------------
# References


## Survey Papers - arxiv.org
* [A Survey of Large Language Models](https://arxiv.org/pdf/2303.18223.pdf)
* [Large Language Models: A Survey](https://arxiv.org/pdf/2402.06196.pdf)


## HuggingFace

* Hugging Face [EPAM profile](https://huggingface.co/epam)
* HuggingFace [course](https://www.youtube.com/watch?v=00GKzGyWFEs&list=PLo2EIpI_JMQvWfQndUesu0nPBAtZ9gP1o) - Technical



## Learnings

* DeepLearning.AI [short courses](https://www.deeplearning.ai/short-courses/)
* 
